In [5]:
import pandas as pd
import os
import numpy as np

In [6]:
def feature_adder(filepath, window_size=10):

    df = pd.read_csv(filepath, sep="\s+", header=None, names=["X", "Y"])

    mean_x = df["X"].mean()
    mean_y = df["Y"].mean()

    new_cols = {}
    prev_x_cols = []
    prev_y_cols= []
    next_x_cols = []
    next_y_cols = []

    for i in range(1, window_size + 1):
        new_cols[f"prev{i}_x"] = df["X"].shift(i)
        new_cols[f"prev{i}_y"] = df["Y"].shift(i)

        new_cols[f"next{i}_x"] = df["X"].shift(-i)
        new_cols[f"next{i}_y"] = df["Y"].shift(-i)
        prev_x_cols.append(f'prev{i}_x')
        prev_y_cols.append(f"prev{i}_y")
        next_x_cols.append(f"next{i}_x")
        next_y_cols.append(f"next{i}_y")
        

    new_cols_df = pd.DataFrame(new_cols, index=df.index)

    for col in new_cols_df.columns:
        if col.endswith("_x"):
            new_cols_df[col].fillna(mean_x, inplace=True)
        else:
            new_cols_df[col].fillna(mean_y, inplace=True)

    df_result = pd.concat([df, new_cols_df], axis=1)

    
    df_result['delta_P'] = (abs(df_result['Y'] - df_result['prev1_y'])).astype(float)
    df_result['delta_T'] = (abs(df_result['X'] - df_result['prev1_x'])).astype(float)
    df_result.loc[0, 'delta_P'] = 0
    df_result.loc[0, 'delta_T'] = 0
    
    
    df_result['avg_y_2n'] = df_result[prev_y_cols + next_y_cols].mean(axis=1)

    

    return df_result

In [7]:
feature_adder('/Users/savinovsvatoslav/Downloads/Task 23/00e4dba2-36d2-42b4-beb1-c55aed75f506').to_csv('proba.csv',index=False)
# feature_adder('/Users/savinovsvatoslav/Code/skvazhina_hack/test')